In [1]:
import os
import torch
import torchvision
import statistics
import copy
import cv2

from PIL import Image
from skimage import measure

import sklearn.metrics as metrics
import numpy as np

import pickle as pkl;

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import util.constants as constants
from util.bbox_methods import (ResNet,
                               bbox_main,
                               gen_state_dict
                              )

In [2]:
model_path = './saved_models/bbox_disease_classifier.pth'
img_path = './beta_test_imgs/00004461_016.png'
transform = torchvision.transforms.ToTensor()

dis_pred_map =  {'Cardiomegaly': 0, 'Effusion': 1, 'Mass': 2, 'Nodule': 3,
                 'Atelectasis': 4, 'Infiltration': 5, 'Pneumonia' : 6, 'Pneumothorax' : 7, 'No Finding': 8}
has_disease_dict = {dis: False for dis in dis_pred_map.keys()}

In [3]:
class_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)

class_model.fc = nn.Sequential(*[
    nn.Linear(in_features=512, out_features=9, bias=True),
    nn.Sigmoid(),
])

class_model = torch.nn.DataParallel(class_model)
class_model.cuda();

class_model.load_state_dict(gen_state_dict(model_path))

Using cache found in /home/npp002/.cache/torch/hub/pytorch_vision_v0.6.0


<All keys matched successfully>

In [4]:
img = Image.open(img_path).convert('RGB')
img = transform(img).unsqueeze(0)

In [5]:
class_model.eval()

with torch.no_grad():
    pred = class_model(img).cpu().numpy()[0]

In [6]:
for d in has_disease_dict.keys():
    pred_index = dis_pred_map[d]
    has_disease_dict[d] = pred[pred_index] > constants.BEST_THRESHOLD[d]

In [7]:
print('---Results---')

for disease, status in has_disease_dict.items():
    print(('{0:15} :').format(disease), status)

---Results---
Cardiomegaly    : True
Effusion        : True
Mass            : False
Nodule          : False
Atelectasis     : True
Infiltration    : True
Pneumonia       : True
Pneumothorax    : False
No Finding      : False
